# Inicialización y funciones

En primer lugar realizamos las importaciones que vamos ha necesitar.

In [18]:
import os

import keras
import numpy as np
from keras.preprocessing import image
from natsort import natsorted
from sklearn import model_selection
from tensorflow.contrib.keras import applications
from tensorflow.contrib.keras import layers, models

Definimos una variables globales que vamos a necesitar.

In [19]:
categories = 'faces/faces/train'
data_path = 'faces/faces'
n_classes = 151

Ordenamos las categorias.

In [20]:
names_categories = natsorted(os.listdir(categories))

Creamos una función que devuelve los datos y sus correspondientes etquetas para la fase de train.

In [21]:
def get_train_data(names_categories, data_path):
    X = []
    y = []
    # Se listan las categorias
    for i, c in enumerate(names_categories):
        # Sobre cada categoria se buscan lso archivos que pertenecesn
        for f in os.listdir(os.path.join(data_path, 'train', c)):
            img_path = os.path.join(data_path, 'train', c, f)
            img = image.load_img(img_path, target_size=(299, 299))
            x = image.img_to_array(img)
            # Se almacena la imagen como datos de entrada y al categoria como etiqueta
            X += [x]
            y += [i]

    y = np.array(y)

    return X, y

Creamos una función que devuelve los datos y sus correspondientes etquetas para la fase de test.

In [22]:
def get_test_data(names_categories, data_path):
    X_test = []
    y_test = []
    # se ordenan los datos de igual manera que las categorias
    for f in natsorted(os.listdir(os.path.join(data_path, 'test'))):
        # A partir del nombre de la imagen se toma la categoria y 
        #se le asigna el mismo numero que en los datos de train
        img_path = os.path.join(data_path, 'test', f)
        name_category = f.split('.')[0]
        label = names_categories.index(name_category)
        # Se almacena la etiqueta en el vector de etiquetas
        y_test.append(label)
        # Se habre la imagen y se guarda en el vector de muestras
        img = image.load_img(img_path, target_size=(299, 299))
        x_test = image.img_to_array(img)

        X_test += [x_test]

    y_test = np.array(y_test)

    return X_test, y_test

# Ejecución

Carga de los datos.

In [23]:
X, y = get_train_data(names_categories, data_path)
X_test, y_test = get_test_data(names_categories, data_path)

Se preprocesan los datos para la que se encuentren acordes a la red ya preentrenada.

In [24]:
X = applications.inception_v3.preprocess_input(np.array(X))
X_test = applications.inception_v3.preprocess_input(np.array(X_test))

KeyboardInterrupt: 

Se carga la red.

In [ ]:
base_model = applications.inception_v3.InceptionV3(input_shape=(299, 299, 3), 
                                                   weights='imagenet', 
                                                   include_top=False)
# add a global spatial average pooling layer
x = layers.GlobalAveragePooling2D()(base_model.output)
# Add the prediction layer of size n_classes
predictions = layers.Dense(n_classes, activation='softmax')(x)
model = models.Model(inputs=base_model.input, outputs=predictions)

Se define que capas son entrenables.

In [ ]:
for layer in base_model.layers:
    layer.trainable = False

Se realiza la separacion entre los datos de train y validación

In [ ]:
x_train, x_val, y_train, y_val = model_selection.train_test_split(X, 
                                                                  y, 
                                                                  test_size=0.2, 
                                                                  random_state=42)

Se define un optimizador y se realiza la compilacion del modelo.

In [ ]:
opt = keras.optimizers.RMSprop(lr=0.1, rho=0.9, epsilon=None, decay=0.0)
model.compile(loss='sparse_categorical_crossentropy', 
              optimizer='rmsprop', 
              metrics=['accuracy'])

Se entrena el modelo.

In [ ]:
model.fit(x=x_train, 
          y=y_train, 
          batch_size=20, 
          epochs=10, 
          validation_data=(x_val, y_val), 
          verbose=2)

Se evaluan los datos de test.

In [ ]:
print(model.evaluate(X_test, y_test))


Ampliación de los datos de entrenamiento mediante el uso de un generador

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
datagen_train = ImageDataGenerator(
                featurewise_center=True,
                featurewise_std_normalization=True,
                rotation_range=20,
                width_shift_range=0.2,
                height_shift_range=0.2,
                horizontal_flip=True
                )

Se entrena el modelo.

In [ ]:
datagen_train.fit(x_train)

model.fit_generator(datagen_train.flow(x_train, y_train, batch_size=20),
                steps_per_epoch=len(x_train) / 10, epochs=20,
                validation_data=(x_val, y_val), validation_steps=20)

Se obtienen valores peores como reconocimiento en el conjunto de validación por lo que el modelo elegido no sería este. Como se puede combrobar con las precisiones obteniddas en train y en validación, el sistema sobreajusta a los datos de entrenamiento.  Pero de todos modos, se evaluan los datos de test.

In [ ]:
print(model.evaluate(X_test, y_test))
